<a href="https://colab.research.google.com/github/BrunoGomesCoelho/mosquito-networking/blob/master/notebooks_tcc/1.7-BrunoGomesCoelho_Colab_fixes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Colab console code
"""
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}setInterval(ClickConnect,60000)
"""

'\nfunction ClickConnect(){\nconsole.log("Working"); \ndocument.querySelector("colab-toolbar-button#connect").click() \n}setInterval(ClickConnect,60000)\n'

## Check tensorflow version

In [0]:
!python3 -m pip install --upgrade tensorflow -q

     |████████████████████████████████| 86.3MB 120kB/s 
     |████████████████████████████████| 3.8MB 30.2MB/s 
     |████████████████████████████████| 450kB 43.1MB/s 


In [0]:
import tensorflow as tf
from packaging import version

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


In [0]:
tf.keras.backend.clear_session()  # For easy reset of notebook state.

In [0]:
# Start logging time
import time
start_time = time.time()

In [0]:
COLAB_IDX = 1
TEMPERATURE = "t02"
FORMAT = "repeat"

In [0]:
SCALE = True
TESTING = True
COLAB = True

if COLAB:
    BASE_DIR = "/content/drive/My Drive/IC/mosquito-networking/"
else:
    BASE_DIR = "../"

In [0]:
if COLAB:
    from google.colab import drive
    drive.flush_and_unmount()
    drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [0]:
if COLAB:
    import sys
    sys.path.append("/content/drive/My Drive/IC/mosquito-networking/")

In [0]:
if COLAB:
    !python3 -m pip install -qr "/content/drive/My Drive/IC/mosquito-networking/drive_requirements.txt"

     |████████████████████████████████| 2.4MB 3.3MB/s 
     |████████████████████████████████| 71kB 4.2MB/s 
     |████████████████████████████████| 122kB 46.0MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 5.3MB 31.6MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 


In [0]:
if not COLAB:
    # OPTIONAL: Load the "autoreload" extension so that code can change
    %load_ext autoreload
    # OPTIONAL: always reload modules so that as you change code in src, it gets loaded
    %autoreload 2

In [0]:
import numpy as np
import pandas as pd
import joblib

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()



- - - 

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Experiment params

In [0]:
# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'epochs': 2 if TESTING else 100}

# load data

In [0]:
# Load scaler
#scaler = joblib.load("../data/interim/scaler.pkl")
if SCALE:
    scaler = joblib.load(BASE_DIR + 
                         f"data/interim/scaler_{FORMAT}_{TEMPERATURE}_no_std.pkl")

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [0]:
#data = np.load(BASE_DIR + f"data/interim/all_wavs_{FORMAT}_{TEMPERATURE}.npz",
data = np.load(BASE_DIR + f"data/interim/all_wavs_{FORMAT}_{TEMPERATURE}_no_std.npz",
               allow_pickle=True)

In [0]:
SCALE=False

In [0]:
# Parse and divide data
train_data = data["train"]
val_data = data["val"]
test_data = data["test"]

if SCALE:
    x_train = scaler.transform(train_data[:, :-1])
    x_val = scaler.transform(val_data[:, :-1])
    x_test = scaler.transform(test_data[:, :-1])
else:
    x_train = train_data[:, :-1]
    x_val = val_data[:, :-1]
    x_test = test_data[:, :-1]

y_train = train_data[:, -1]
y_val = val_data[:, -1]
y_test = test_data[:, -1]

In [0]:
# Reshape data
x_train = x_train.reshape((*x_train.shape, 1))
x_val = x_val.reshape((*x_val.shape, 1))
x_test = x_test.reshape((*x_test.shape, 1))

In [0]:
# Generators
# I am not using generators since currently the dataset fits on RAM

# Load model

In [0]:
"""A much better way to load a custom module since it allows us to change it easierly
    on the fly. Idea from https://zerowithdot.com/colab-workspace/
"""
from os.path import join
from importlib.machinery import SourceFileLoader
wingbeats_model = SourceFileLoader('wingbeats_model', join(BASE_DIR, 'src/models/wingbeats_model.py')).load_module()
model = wingbeats_model.WingbeatsNetModel(dropout=False)
#from src.models.wingbeats_model import WingbeatsNetModel
#model = WingbeatsNetModel(dropout=False)
# load weights?
#f"runs/colab/{COLAB_IDX-1}/model_epoch_90.pt"))
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise (GaussianNois (None, 11025, 1)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 11023, 16)         64        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 11021, 16)         784       
_________________________________________________________________
batch_normalization (BatchNo (None, 11021, 16)         64        
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 929
Trainable params: 897
Non-trainable params: 32
_________________________________________________________

![](wingbeats_model.png)

<br>
<br>
<br>
<br>
<br>
<br>

# Start tensorboard

In [0]:
save_path =  BASE_DIR + f"runs/TCC/{TEMPERATURE}/{FORMAT}/{COLAB_IDX}/"

In [0]:
!python3 -m pip install --upgrade grpcio

Requirement already up-to-date: grpcio in /usr/local/lib/python3.6/dist-packages (1.24.3)


In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
#!kill 5269
%tensorboard --logdir logs

/bin/bash: line 0: kill: (5269) - No such process


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# train function

In [0]:
MODEL = "testing"
model_name = f"{MODEL}"

In [0]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping, 
                                        ReduceLROnPlateau, CSVLogger,
                                        TensorBoard)

In [0]:
# Overfit small amount of data
temp = x_train.copy()
temp_y = y_train.copy()

In [0]:
x_train = x_train[:10]
y_train = y_train[:10]

params["epochs"] = 10
params["batch_size"] = 20

model = wingbeats_model.WingbeatsNetModel(dropout=False, batch_norm=False, 
                                          use_noise=False, blocks=3)#lr=0.0001)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 11023, 16)         64        
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 11021, 16)         784       
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 11019, 32)         1568      
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 11017, 32)         3104      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 5508, 32)          0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 5506, 64)          6208      
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 5504, 64)         

In [0]:
from datetime import datetime

top_weights_path = 'model_' + model_name + '.h5'

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir, write_grads=True,
                                   write_graph=True, write_images=True,
                                   histogram_freq=1)

callbacks_list = [
    #ModelCheckpoint(top_weights_path, monitor='val_acc', verbose=1,
    #                save_best_only=True, save_weights_only=True),
    #EarlyStopping(monitor='val_accuracy', patience=6, verbose=1),
    tensorboard_callback]#,
    #ReduceLROnPlateau(monitor = 'val_accuracy', factor=0.1, 
    #                  patience=3, verbose=1),
    #CSVLogger('model_' + model_name + '.log')]

histories = model.fit(x_train, y_train, 
          validation_data=[x_val, y_val],
          callbacks=callbacks_list,
          **params)

#model.load_weights(top_weights_path)
#loss, acc = model.evaluate(x_test, y_test)

#print('loss', loss)
#print('Test accuracy:', acc)

Train on 10 samples, validate on 743 samples
Epoch 1/100
10/10 [==============================] - 12s 1s/sample - loss: 12.8364 - accuracy: 0.5000 - val_loss: 0.7866 - val_accuracy: 0.4616
Epoch 2/100
10/10 [==============================] - 11s 1s/sample - loss: 0.8034 - accuracy: 0.5000 - val_loss: 3.1100 - val_accuracy: 0.4670
Epoch 3/100
10/10 [==============================] - 11s 1s/sample - loss: 3.1343 - accuracy: 0.5000 - val_loss: 1.8146 - val_accuracy: 0.5330
Epoch 4/100
10/10 [==============================] - 11s 1s/sample - loss: 1.5043 - accuracy: 0.5000 - val_loss: 1.9336 - val_accuracy: 0.5330
Epoch 5/100
10/10 [==============================] - 11s 1s/sample - loss: 1.6035 - accuracy: 0.5000 - val_loss: 0.7300 - val_accuracy: 0.4603
Epoch 6/100
10/10 [==============================] - 11s 1s/sample - loss: 0.7376 - accuracy: 0.5000 - val_loss: 1.2362 - val_accuracy: 0.4670
Epoch 7/100
10/10 [==============================] - 11s 1s/sample - loss: 1.2356 - accuracy: 0.

In [0]:
def check_grad_proportion(model, x_train, layer_idx=0):
    name = model.get_config()["layers"][layer_idx]["config"]["name"]
    inputs = tf.cast(x_train, tf.float32)

    with tf.GradientTape() as tape:
        preds = model(inputs)

    grads = tape.gradient(preds, model.get_layer(name).trainable_variables)

    weights =  model.get_layer(name).weights
    proportion = grads[0] / weights[0]
    proportion_avg = tf.math.reduce_mean(proportion)
    print(f"Average proportion between grad and weights: {proportion_avg}")
    print(f"Some proportions:")
    print(proportion[:5])
    return grads, weights


In [0]:
grads, weights = check_grad_proportion(model, x_train)

Average proportion between grad and weights: 68.9854736328125
Some proportions:
tf.Tensor(
[[[ 7.6938620e+00 -5.7699676e+00 -3.3229630e+01 -1.7223685e+02
    4.4788728e+00  6.2952038e+01  2.5098743e+01 -1.0169546e+01
   -1.6430066e+00 -1.1519778e+03  2.4968575e+02 -1.1131790e+01
    9.3059874e+00  3.7635372e+01 -1.4491734e+02  1.4018068e+01]]

 [[ 4.3587241e+00  2.5055573e+01 -4.6687897e+02 -1.4131907e+01
    4.9236407e+00  9.2895287e+01 -1.8585780e+02 -3.6796679e+00
    5.3466258e+00  1.8454512e+02  5.0662224e+01  3.0264890e+02
    2.2525251e+01 -4.3277748e+01  4.9968178e+01 -2.3169279e+01]]

 [[ 2.2614448e+00 -7.4580505e+01 -1.5991063e+02 -1.2604927e+01
   -2.8860109e+00 -4.3903851e+01  4.0538604e+03  2.3806844e+00
    1.1009400e+00 -8.2831230e+01  6.0226599e+02  2.6159475e+01
   -2.3446371e+01  7.9546104e+01  2.6068020e+01  3.2096184e+01]]], shape=(3, 1, 16), dtype=float32)
